# Катим питон в прод на докере и анаконде

Мы хотим выкатить в прод вот такую простую приложулю:

```python
from flask import Flask
import os

app = Flask(__name__)


@app.route("/")
def hello():
    return f"Hello, {os.environ.get('user', 'world')}!"
```

Она не тянет на то, чтобы деплоить её в отказоустойчивый кластер, поэтому ограничимся только докером :)

## Управление окружениями в conda

### Создаём окружение в conda

Для нового проекта всегда создаём новое окружение. Это позволит нам ставить только те пакеты, которые нужны для текущего проекта. Предыдущие проекты не сломаются, какие бы версии пакетов мы не поставили. Ещё одно преимущество `conda` перед `pip` в том, что конда ставит не только питоновские пакеты, но и системные библиотеки, от которых они могут зависеть. Например, вместе с `numpy` сразу ставятся `MKL` или `OpenBLAS`, которые сильно ускоряют векторные операции. Если ставить `numpy` через `pip`, то эти библы придётся ставить отдельно ручками.

Необязательно ставить полную анаконду. Достаточно поставить Miniconda: https://docs.conda.io/en/latest/miniconda.html. Она отличается от полного дистра только тем, что в неё не включены бинарники кучи библиотек. Всё равно мы потом скачаем их из интернета.

Перед тем, как создавать окружения, добавим канал Conda Forge. В этом канале гораздо больше пакетов, чем в дефолтном, и он поддерживается большим комьюнити.

```bash
conda config --add channels conda-forge
conda config --set channel_priority strict 
```

Создадим окружение и активируем его:

```bash
conda create -n sandbox python=3.7.3
conda activate sandbox
```

Допустим, что мы хотим сделать простую приложулю на Flask. Поставим его.

```bash
conda install flask -y
```

### Создаём спецификацию окружения

Одна из задач окружения — быть воспроизведённым на машинах других разработчиков и в проде. При создании нового окружения мы можем указать файл спецификации, и конда поставит все перечисленные в нём пакеты. А ещё мы можем экспортировать спецификацию любого существующего окружения.

Экспортнём спеку от нашего окружения:

```bash
conda env export --no-builds --ignore-channels -n sandbox -f environment.yml
```

Посмотрим, что получилось:

```yaml
name: sandbox
channels:
  - conda-forge
  - defaults
dependencies:
  - ca-certificates=2019.9.11
  - certifi=2019.6.16
  - click=7.0
  - flask=1.1.1
  - itsdangerous=1.1.0
  - jinja2=2.10.3
  - markupsafe=1.1.1
  - openssl=1.1.1c
  - pip=19.3.1
  - python=3.7.3
  - setuptools=41.4.0
  - sqlite=3.30.1
  - vc=14.1
  - vs2015_runtime=14.16.27012
  - werkzeug=0.16.0
  - wheel=0.33.6
  - wincertstore=0.2
prefix: C:\tools\miniconda3\envs\sandbox
```

Видим, что конда радостно слила нам все пакеты, которые есть в окружении. При этом, у нас есть пакеты, которые явно относятся только к Windows: `vc`, `vs2015_runtime` и т. д. Чтобы спека была портабельно между операционками, с ней нужно работать так:

1. В начале проекта ставим в окружение все библы, которые, на наш взгляд нам понадобятся.
2. Экспортируем спеку окружения средствами конды.
3. Вручную удаляем из файла:
    * `prefix`, т. к. путь к окружению на каждый инсталляции будет свой.
    * Все пакеты, кроме пакетов первого уровня.
    * Версии всех пакетов, которые мы явно не хотим зафиксировать на определённой версии.
4. Когда в ходе проекта нам понадобится поставить новые пакеты, мы их ставим как обычно через `conda install` и руками дописываем в спеку.

#### Что значит «пакеты верхнего уровня»?

Когда мы планировали нашу приложуху, мы решили, что нам нужен Питон и flask. Явным образом мы ставили только эти библы; всё остальное — подтянутые зависимости. Вот как раз эти зависимости нам и нужно удалить из спеки, так как под разными ОС они могут быть разными. Просто скажем Конде, что нам нужен Питон и flask, а она сама подтянет нужные по ситуации зависимости.

Если не помните, какие пакеты у вас на верхнем уровне, можно поставить `conda-tree` и посмотреть:

```bash
conda install -n base conda-tree
conda-tree -n sandbox leaves
>> ['flask']
```

Режим `leaves` показывает листья графа зависимостей в заданном окружении. В нашем случае нам показали только flask, что логично. Значит, оставляем в окружении только Питон и flask:

```yaml
name: sandbox
channels:
  - conda-forge
  - defaults
dependencies:
  - flask
  - python=3.7.3
```

При этом, мы указали версию только у Питона. Нам, по большому счёту, всё равно, какой версии будет flask, т. к. мы пишем простую приложуху. Идея в том, чтобы пинить версии только тех пакетов, специфичные версии которых вас действительно волнуют.

### Создаём окружение по спеке

Теперь мы можем спокойно создать новое окружение по спеке, и оно создастся нормально на любой ОС:

```bash
conda env create -n sandbox2 --file environment.yml
```

## Docker

### Краткое введение в Docker

Docker предоставляет легковесный слой изоляции для приложений под Linux. В отличие от традиционной виртуальной машины, Docker не эмулирует железо и не поднимает полноценную гостевую ОС для каждого контейнера. Вместо этого, он шарит ядро Linux на хосте и с помощью хитрых хаков добивается полной изоляции контейнеров друг от друга. Приложения внутри контейнера думают, что они крутятся на отдельной машине.

![](img/docker_vs_vm.png)

#### Images and containers

Есть два основых понятия: `Image` и `Container`. Можно сказать, что image — это описание класса, а контейнер — его инстась. Мы заранее создаём image — замороженный снимок файловой системы «гостевой» ОС. После этого мы можем запустить сколько угодно контейнеров на основе одного образа, и все они могут работать по-разному.

![](img/sharing-layers.jpg)

### Dockerfile и слои

Образ описывается в Dockerfile. Каждая инструкция в Dockerfile создаёт новый слой в образе. Слой — это то же самое, что коммит в Git. Слой сохраняет только те изменения, которые произошли после выполнения команды относительно предыдущего слоя.

Посмотрим на Dockerfile и разберёмся, что происходит.

```dockerfile
FROM olegtarasov/miniconda3-forge

WORKDIR /usr/app

COPY environment.yml .

RUN conda env create -n sandbox -f environment.yml && \
    conda clean --all -y

COPY . .

CMD ["/bin/bash", "-l", "-c", "conda activate sandbox && env FLASK_APP=hello.py flask run --host=0.0.0.0"]
```

Первая инструкция — `FROM` говорит о том, с какой базы мы начинаем строить контейнер. База — это точно такой же образ, который кто-то заранее сделал и опубликовал. Можно строить контейнеры с голого линукса. В нашем примере мы начинаем с образа, в котором уже есть Miniconda.

Далее мы говорим, что все последующие команды будут выполнятся в контексте директории `/usr/app` внутри контейнера.

После этого мы копируем нашу спецификацию окружения в контейнер. Мы подразумеваем, что билд образа мы запускаем из папки с нашим проектом, поэтому путь `src` у нас без указания директории, а в `dst` мы ставим `.` потому что ранее сменили «рабочую» директорию контейнера на `/usr/app`.

Далее с помощью команды `RUN` мы создаём новое окружение и подчищаем бинарные дисты анаконды, чтобы экономить место.

Разберёмся более подробно, что здесь происходит. Во время билда образа Docker по-настоящему запускает Linux в изолированном окружении и выполняет все команды, которые мы пишем в Dockerfile. Таким образом, здесь мы действительно создадим новое окружение по нашей спеке. После окончания выполнения команды Docker посмотрит, что изменилось в файловой системе образа, и запишет изменения как новый слой.

После этого мы копируем оставшиеся файлы приложения в наш контейнер. Почему не скопировать сразу всё? Ради оптимизации билда. Спека окружения меняется реже, чем исходники, поэтому слои с `COPY environment.yml .` и `RUN ...` Docker может закэшировать и не выполнять их, если не изменилась спека.

Последней командой мы указываем, что должно выполняться, когда мы создадим из нашего образа контейнер и запустим его. Здесь мы запускаем `bash` в режиме логина, чтобы можно было активировать окружение conda; активируем окружение и запускаем нашу простую приложуху.

### Собираем и тестируем образ

Перейдём в директорию с нашим докерфайлом и соберём образ. Сразу присвоим ему тэг, чтобы не разбираться с sha-хэшами.

```bash
docker build . -t olegtarasov/hello
```

Убедимся, что образ есть:

```bash
docker image ls
>> REPOSITORY                     TAG                 IMAGE ID            CREATED             SIZE
>> olegtarasov/hello              latest              1c01b095f365        2 minutes ago       586MB
```

Теперь давайте создадим одноразовый контейнер и запустим его.

```bash
docker run -it --rm olegtarasov/hello
```

Опции `-it` говорят, что мы хотим запуcтить интерактивную сессию и прицепить консоль ОС внутри контейнера к нашей текущей консоли. `--rm` говорит о том, что контейнер будет удалён сразу после остановки.

После запуска контейнера мы увидим, что наш код работает, и простой сервер висит на порту 5000. Но есть проблема: это порт ОС внутри контейра, он не проброшен наружу. Чтобы достучаться до него с хоста, нужно пробросить этот порт из контейнера наружу. Давайте заодно создадим постоянный контейнер с каким-нибудь именем и запустим его.

```bash
docker create --name hello1 -p 5000:5000 olegtarasov/hello
docker start hello1
```

Командой `create` мы создали контейнер `hello1` из нашего образа, и запустили его командой `start`. Откроем http://localhost:5000/, и увидим, что нам выводится `Hello, world!`.

Помните, что в исходниках мы читаем имя из переменной окружения? Давайте создадим из нашего образа ещё один контейнер, и зададим эту переменную:

```bash
docker create --name hello2 -p 5001:5000 -e user=John olegtarasov/hello
docker start hello2
```

Перейдём на http://localhost:5001/, и увидим `Hello, John!`. Посмотрим на наши контейнеры:

```bash
docker ps
>> CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                    NAMES
>> cdd8b74ef050        olegtarasov/hello   "/tini -- /bin/bash …"   57 seconds ago      Up 50 seconds       0.0.0.0:5001->5000/tcp   hello2
>> 59bddc00471c        olegtarasov/hello   "/tini -- /bin/bash …"   6 minutes ago       Up 6 minutes        0.0.0.0:5000->5000/tcp   hello1
```

Мы видим, что у нас запущено два контейнера. Оба были созданы из одного и того же образа, но работают они полностью независимо, висят на разных портах и выдают разный результат.